In [1]:
import pandas as pd
import plotly as py
import plotly.graph_objects as go
import plotly.express as px 
import ipywidgets as widgets
import numpy as np
from scipy import special
import time
import math as ms
import sys

from IPython.display import display, Markdown
from ipywidgets import Layout

sys.path.insert(0, '..\..\Resources\Python-Functions')
import BigQuery
 
client = BigQuery.getBigQueryClient_TDMScenarios()

In [2]:
# include in all scenario groups
lstIncludeInAll = ['BY','TIP']

# dataframe to create subcategories
dfModeGroups = pd.DataFrame([
    ['Walk'     ,'1: Non-Motorized'         ],
    ['Bike'     ,'1: Non-Motorized'         ],
    ['Walk'     ,'1a: Walk'                 ],
    ['Bike'     ,'1b: Bike'                 ],
    ['Auto'     ,'2: Auto'                  ],
    ['SchoolBus','3: SchoolBus'             ],
    ['LCL'      ,'4: Transit'               ],
    ['COR'      ,'4: Transit'               ],
    ['EXP'      ,'4: Transit'               ],
    ['BRT'      ,'4: Transit'               ],
    ['LRT'      ,'4: Transit'               ],
    ['CRT'      ,'4: Transit'               ],
    ['LCL'      ,'4a: Local Bus'            ],
    ['COR'      ,'4b: Core Bus'             ],
    ['EXP'      ,'4c: Express Bus'          ],
    ['BRT'      ,'4d: Bus-Rapid Transit'    ],
    ['LRT'      ,'4e: Light-Rail Transit'   ],
    ['CRT'      ,'4f: Commuter-Rail Transit'],
    ['Walk'     ,'0: Total'                 ],
    ['Bike'     ,'0: Total'                 ],
    ['Auto'     ,'0: Total'                 ],
    ['SchoolBus','0: Total'                 ],
    ['LCL'      ,'0: Total'                 ],
    ['COR'      ,'0: Total'                 ],
    ['EXP'      ,'0: Total'                 ],
    ['BRT'      ,'0: Total'                 ],
    ['LRT'      ,'0: Total'                 ],
    ['CRT'      ,'0: Total'                 ]]
,columns=('MODE','modeGroup'))
#dfModeGroups

In [3]:
# FILTER
#strSQLWhere = ' WHERE NOT (t.scenarioID = 33)'
strSQLWhere = ''

# merge scenario data onto transit share data

# read transit summary from biq query
dfTransitSummary = client.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.transit_share AS t" + strSQLWhere).to_dataframe()
#display(dfTransitSummary)

dfRouteSummary = client.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.route_summary AS t" + strSQLWhere).to_dataframe()
#display(dfRouteSummary)

# read scenarios data biq query
dfScenarios = client.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.scenarios AS t" + strSQLWhere).to_dataframe()
#display(dfScenarios)

# merge two dataframes
dfTransitSummaryWithScenarioData = pd.DataFrame.merge(dfScenarios,dfTransitSummary,on='scenarioID')
#display(dfTransitSummaryWithScenarioData)

# merge two dataframes
dfRouteSummaryWithScenarioData = pd.DataFrame.merge(dfScenarios,dfRouteSummary,on='scenarioID')
#display(dfTransitSummaryWithScenarioData)

# merge to mode groupings dataframe
dfTransitSummaryPlotData = pd.DataFrame.merge(dfTransitSummaryWithScenarioData,dfModeGroups,on='MODE')
#dfTransitSummaryPlotData


In [4]:
dfScenarios

,scenarioID,tdmVersion,tdmVersionWithDate,scenarioName,scenarioGroup,scenarioYear
0,3.01,v9,WF TDM v9.0 - 2022-12-19,BY_2019,BY,2019
1,4.01,v9,WF TDM v9.0 - 2023-02-23-MCCalib-r0,BY_2019,BY,2019
2,5.01,v9,WF TDM v9.0 - 2023-02-23-MCCalib-r2,BY_2019,BY,2019
3,7.01,v9,WF TDM v9.0 - 2023-04-14,BY_2019,BY,2019
4,8.01,v9,WF TDM v9.0 - 2023-04-28,BY_2019,BY,2019
...,...,...,...,...,...,...
66,2.07,v832,WF TDM v8.3.2 - 2022-09-21,Needs_2042,Needs,2042
67,2.08,v832,WF TDM v8.3.2 - 2022-09-21,Needs_2050,Needs,2050
68,2.09,v832,WF TDM v8.3.2 - 2022-09-21,NoBuild_2032,NoBuild,2032
69,2.10,v832,WF TDM v8.3.2 - 2022-09-21,NoBuild_2042,NoBuild,2042


In [5]:
# check to see if all scenarios has same record count
dfTransitSummaryPlotData_Check = dfTransitSummaryPlotData
dfTransitSummaryPlotData_Check.groupby(['tdmVersionWithDate','scenarioGroup','scenarioYear']).agg(COUNT=('TRIPS','count'))

COUNT
tdmVersionWithDate          scenarioGroup scenarioYear       
WF TDM v8.3.2 - 2022-02-04a BY            2019            280
                            RTP           2030            280
                                          2040            280
                                          2050            280
                            TIP           2024            280
...                                                       ...
WF TDM v9.0 - 2023-06-30    NoBuild       2050            280
                            RTP           2032            280
                                          2042            280
                                          2050            280
                            TIP           2028            280

[70 rows x 1 columns]

#### Check BY_2019 between WF TDM v9.0 - 2023-04-14 has the same results as WF TDM v9.0 - 2023-04-14-CRLF-run

In [6]:
#dfTransitSummaryPlotData_2019 = dfTransitSummaryPlotData[dfTransitSummaryPlotData['scenarioID'].isin([7.01,8.01])]
#fig = px.bar(dfTransitSummaryPlotData_2019, x='tdmVersionWithDate', y='TRIPS', color='MODE')
#fig.show()

In [7]:
# Transit Split
#dfTransitSummaryPlotData_2019_Transit = dfTransitSummaryPlotData_2019[dfTransitSummaryPlotData_2019['MODE'].isin(['BRT','COR','CRT','EXP','LCL','LRT'])]
#fig = px.bar(dfTransitSummaryPlotData_2019_Transit, x='tdmVersionWithDate', y='TRIPS', color='MODE')
#fig.show()

#### Plotting Function

In [8]:
#PLOTTING FUNCTION

import math

def update_plot(tdmVersionsWithDate, scenarioGroups, modeGroups, trippurps, periods):
    
    #output.clear_output()

    global firstTime
    if firstTime:
        firstTime = False
    else:
            
        data = []

        for v in tdmVersionsWithDate:
            for g in scenarioGroups:
                for m in modeGroups:

                    # only do if data in dataframe since BY data is concatonated later
                    if dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==v) & (dfTransitSummaryPlotData['scenarioGroup'].isin([g])) & (dfTransitSummaryPlotData['modeGroup']==m) & (dfTransitSummaryPlotData['TRIPPURP'].isin(trippurps)) & (dfTransitSummaryPlotData['PERIOD'].isin(periods))].shape[0]>1:

                        # data for plotting from filtered dataframe}
                        plotdata = dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==v                         ) &
                                                            (dfTransitSummaryPlotData['scenarioGroup'     ].isin(lstIncludeInAll + [g])) &
                                                            (dfTransitSummaryPlotData['modeGroup'         ]==m                         ) &
                                                            (dfTransitSummaryPlotData['TRIPPURP'          ].isin(trippurps)            ) &
                                                            (dfTransitSummaryPlotData['PERIOD'            ].isin(periods)              )]

                        #display(plotdata)

                        plotdata = plotdata.groupby(['scenarioYear'], as_index=False).agg(TRIPS=('TRIPS','sum'))

                        # fill any NaN values with zeros
                        plotdata = plotdata.fillna(0)

                        #display(plotdata)

                        xplot = plotdata['scenarioYear']
                        yplot = plotdata['TRIPS'       ]

                        trace1 = go.Scatter(
                            x=xplot,
                            y=yplot,
                            mode='markers+lines',
                            name= v + ' ' + g + ' ' + m,
                            marker=dict(size=12,
                                    line=dict(width=2,
                                                color='DarkSlateGrey'))#,
                            #stackgroup='one',
                            #groupnorm='percent' # sets the normalization for the sum of the stackgroup
            #                line=dict(
            #                    shape='spline'
            #                )
                        )
                        data.append(trace1)


        layout = go.Layout(
            title='Trips by Mode (' + '/'.join(trippurps) + ' ' + '/'.join(periods) + ')',
            yaxis=dict(
                title='Trips',
                rangemode = 'tozero'#,
                #range=(0,np.null)
            ),
            xaxis=dict(
                title='Year',
                range=(2018,2051)
            ),
            width=1000,
            height=450
        )
    
        fig = go.Figure(data=data, layout=layout)
        py.offline.iplot(fig)

In [9]:
# MAKE INTERACTIVE CHART
py.offline.init_notebook_mode(connected=True)

lstTdmVersionWithDate = sorted(dfTransitSummaryPlotData['tdmVersionWithDate'].unique().tolist())
lstScenarioGroups     = sorted(dfTransitSummaryPlotData['scenarioGroup'     ].unique().tolist())
lstModes              = sorted(dfTransitSummaryPlotData['modeGroup'         ].unique().tolist())
lstTripPurp           = sorted(dfTransitSummaryPlotData['TRIPPURP'          ].unique().tolist())
lstPeriod             = sorted(dfTransitSummaryPlotData['PERIOD'            ].unique().tolist())

# remove include in all list
lstScenarioGroups = [i for i in lstScenarioGroups if i not in lstIncludeInAll] 

selectTdmVersionsWithDate = widgets.SelectMultiple(options=lstTdmVersionWithDate, value=(lstTdmVersionWithDate), description='TDM Version'   )
selectScenarioGroups      = widgets.SelectMultiple(options=lstScenarioGroups    , value=('RTP',               ), description='Scenario Group')
selectModes               = widgets.SelectMultiple(options=lstModes             , value=('4: Transit',        ), description='Mode Group'    )
selectTripPurps           = widgets.SelectMultiple(options=lstTripPurp          , value=(lstTripPurp          ), description='Purpose'       )
selectPeriods             = widgets.SelectMultiple(options=lstPeriod            , value=(lstPeriod            ), description='Periods'       )

# Set up a global variable to track whether the widgets have been changed
firstTime = True

# create output widget to display filtered DataFrame
output = widgets.Output()
hbox = widgets.HBox([selectTdmVersionsWithDate, selectScenarioGroups, selectModes, selectTripPurps, selectPeriods])

# create interactive widget
interactive_output = widgets.interactive_output(update_plot, {'tdmVersionsWithDate':selectTdmVersionsWithDate, 'scenarioGroups':selectScenarioGroups, 'modeGroups':selectModes, 'trippurps':selectTripPurps, 'periods':selectPeriods})

# Display a markdown heading
display(Markdown('# Trip Mode Split Interactive Chart'))
display(hbox)
display(interactive_output)
display(output)

#ADD RAW OBSERVED CURVES

# Trip Mode Split Interactive Chart

Output()

Output()

In [16]:
#PLOTTING FUNCTION

import math

def update_plot_stackedarea(tdmVersionWithDate, scenarioGroup, modeGroups, trippurps, periods, percentOrTotal):
    
    output.clear_output()

    global firstTime
    if firstTime:
        firstTime = False
    else:
        
        data = []

        modeGroups = sorted(modeGroups)

        for m in modeGroups: 
            # only do if data in dataframe since BY data is concatonated later
            if dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==tdmVersionWithDate) &
                                        (dfTransitSummaryPlotData['scenarioGroup'     ]==scenarioGroup     ) &
                                        (dfTransitSummaryPlotData['modeGroup'         ]==m                 ) &
                                        (dfTransitSummaryPlotData['TRIPPURP'          ].isin(trippurps)    ) &
                                        (dfTransitSummaryPlotData['PERIOD'            ].isin(periods)      )].shape[0]>1:

                # data for plotting from filtered dataframe}
                plotdata = dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==tdmVersionWithDate                    ) &
                                                    (dfTransitSummaryPlotData['scenarioGroup'     ].isin(lstIncludeInAll + [scenarioGroup])) &
                                                    (dfTransitSummaryPlotData['modeGroup'         ]==m                                     ) &
                                                    (dfTransitSummaryPlotData['TRIPPURP'          ].isin(trippurps)                        ) &
                                                    (dfTransitSummaryPlotData['PERIOD'            ].isin(periods)                          )]

                #display(plotdata)

                plotdata = plotdata.groupby(['scenarioYear'], as_index=False).agg(TRIPS=('TRIPS','sum'))

                # fill any NaN values with zeros
                plotdata = plotdata.fillna(0)

                #display(plotdata)

                xplot = plotdata['scenarioYear']
                yplot = plotdata['TRIPS'       ]

                trace1 = go.Scatter(
                    x=xplot,
                    y=yplot,
                    mode='lines',
                    name= m,
                    stackgroup='one'#,
                    #groupnorm='percent' # sets the normalization for the sum of the stackgroup
                )
                if percentOrTotal=='Share':
                    trace1.groupnorm = 'percent'
                data.append(trace1)


        layout = go.Layout(
            title=tdmVersionWithDate + ' Trips Mode Split (' + '/'.join(trippurps) + ' ' + '/'.join(periods) + ')',
            yaxis=dict(
                title='Trips'#,
                #rangemode = 'tozero',
                #ticksuffix='%',
                #range=(0,100)
            ),
            xaxis=dict(
                title='Year'#,
                #range=(2018,2051)
            ),
            width=1000,
            height=600
        )
        
        fig2 = go.Figure(data=data, layout=layout)
        py.offline.iplot(fig2)
        #display(fig2)

        #widgets_changed = False



In [18]:
# MAKE INTERACTIVE CHART
#py.offline.init_notebook_mode(connected=True)

lstTdmVersionsWithDate = sorted(dfTransitSummaryPlotData['tdmVersionWithDate'].unique().tolist())
lstScenarioGroups      = sorted(dfTransitSummaryPlotData['scenarioGroup'     ].unique().tolist())
lstModes               = sorted(dfTransitSummaryPlotData['modeGroup'         ].unique().tolist())
lstTripPurp            = sorted(dfTransitSummaryPlotData['TRIPPURP'          ].unique().tolist())
lstPeriod              = sorted(dfTransitSummaryPlotData['PERIOD'            ].unique().tolist())

# remove include in all list
lstScenarioGroups = [i for i in lstScenarioGroups if i not in lstIncludeInAll] 

selectTdmVersionWithDate = widgets.Select        (options=lstTdmVersionWithDate, value=(lstTdmVersionsWithDate[2]), description='TDM Version'   , layout=Layout(display="flex", flex_flow='row'))
selectScenarioGroup      = widgets.Select        (options=lstScenarioGroups    , value=('RTP'                    ), description='Scenario Group', layout=Layout(display="flex", flex_flow='row'))
selectModes              = widgets.SelectMultiple(options=lstModes             , value=(lstModes[7:13]           ), description='Mode Group'    , layout=Layout(display="flex", flex_flow='row'))
selectTripPurps          = widgets.SelectMultiple(options=lstTripPurp          , value=(lstTripPurp              ), description='Purpose'       , layout=Layout(display="flex", flex_flow='row'))
selectPeriods            = widgets.SelectMultiple(options=lstPeriod            , value=(lstPeriod                ), description='Periods'       , layout=Layout(display="flex", flex_flow='row'))
selectPercentOrTotal     = widgets.Select        (options=['Share','Total']    , value=('Share'                  ), description='Stack Type'    , layout=Layout(display="flex", flex_flow='row'))

# Set the width of the widget
selectTdmVersionWithDate.layout.width = '350px'
selectScenarioGroup     .layout.width = '150px'
selectModes             .layout.width = '250px'
selectTripPurps         .layout.width = '150px'
selectPeriods           .layout.width = '120px'
selectPercentOrTotal    .layout.width = '180px'

# Set up a global variable to track whether the widgets have been changed
firstTime = True

# create output widget to display filtered DataFrame
output = widgets.Output()
hbox = widgets.HBox([selectTdmVersionWithDate, selectScenarioGroup, selectModes, selectTripPurps, selectPeriods, selectPercentOrTotal])

# create interactive widget
interactive_output = widgets.interactive_output(update_plot_stackedarea, {'tdmVersionWithDate': selectTdmVersionWithDate, 'scenarioGroup':selectScenarioGroup, 'modeGroups':selectModes, 'trippurps':selectTripPurps, 'periods':selectPeriods, 'percentOrTotal':selectPercentOrTotal})

# Display a markdown heading
display(Markdown('# Trip Mode Split Interactive Chart'))
display(hbox)
display(interactive_output)
display(output)
#ADD RAW OBSERVED CURVES

# Trip Mode Split Interactive Chart

Output()

Output()

# Boardings

In [19]:
#PLOTTING FUNCTION

import math

def update_plot_stackedarea_boardings(tdmVersionWithDate, scenarioGroup, percentOrTotal):
    
    output.clear_output()

    global firstTime
    if firstTime:
        firstTime = False
    else:
            
        data = []

        modes = sorted(dfRouteSummaryWithScenarioData['MODE'].unique())

        for m in modes: 
            # only do if data in dataframe since BY data is concatonated later
            if dfRouteSummaryWithScenarioData[(dfRouteSummaryWithScenarioData['tdmVersionWithDate']==tdmVersionWithDate) &
                                            (dfRouteSummaryWithScenarioData['scenarioGroup'     ]==scenarioGroup     ) &
                                            (dfRouteSummaryWithScenarioData['MODE'              ]==m                 )].shape[0]>1:

                # data for plotting from filtered dataframe}
                plotdata = dfRouteSummaryWithScenarioData[(dfRouteSummaryWithScenarioData['tdmVersionWithDate']==tdmVersionWithDate                    ) &
                                                        (dfRouteSummaryWithScenarioData['scenarioGroup'     ].isin(lstIncludeInAll + [scenarioGroup])) &
                                                        (dfRouteSummaryWithScenarioData['MODE'              ]==m                                     )]

                #display(plotdata)

                plotdata = plotdata.groupby(['scenarioYear'], as_index=False).agg(BOARDINGS=('DY_BRDA','sum'))

                # fill any NaN values with zeros
                plotdata = plotdata.fillna(0)

                #display(plotdata)

                xplot = plotdata['scenarioYear']
                yplot = plotdata['BOARDINGS'   ]

                trace1 = go.Scatter(
                    x=xplot,
                    y=yplot,
                    mode='lines',
                    name= 'Mode ' + str(m),
                    stackgroup='one'#,
                    #groupnorm='percent' # sets the normalization for the sum of the stackgroup
                )
                if percentOrTotal=='Share':
                    trace1.groupnorm = 'percent'
                data.append(trace1)


        layout = go.Layout(
            title=tdmVersionWithDate + ' Boardings by Mode',
            yaxis=dict(
                title='Trips'#,
                #rangemode = 'tozero',
                #ticksuffix='%',
                #range=(0,100)
            ),
            xaxis=dict(
                title='Year'#,
                #range=(2018,2051)
            ),
            width=1000,
            height=400
        )
        
        fig2 = go.Figure(data=data, layout=layout)
        py.offline.iplot(fig2)


In [20]:
# MAKE INTERACTIVE CHART
py.offline.init_notebook_mode(connected=True)

from ipywidgets import Layout

lstTdmVersionsWithDate = sorted(dfRouteSummaryWithScenarioData['tdmVersionWithDate'].unique().tolist())
lstScenarioGroups      = sorted(dfRouteSummaryWithScenarioData['scenarioGroup'     ].unique().tolist())

# remove include in all list
lstScenarioGroups = [i for i in lstScenarioGroups if i not in lstIncludeInAll] 

selectTdmVersionWithDate = widgets.Select        (options=lstTdmVersionWithDate, value=(lstTdmVersionsWithDate[2]), description='TDM Version'   , layout=Layout(display="flex", flex_flow='row'))
selectScenarioGroup      = widgets.Select        (options=lstScenarioGroups    , value=('RTP'                    ), description='Scenario Group', layout=Layout(display="flex", flex_flow='row'))
selectPercentOrTotal     = widgets.Select        (options=['Share','Total']    , value=('Share'                  ), description='Stack Type'    , layout=Layout(display="flex", flex_flow='row'))

# create output widget to display filtered DataFrame
output = widgets.Output()
hbox = widgets.HBox([selectTdmVersionWithDate, selectScenarioGroup])


# Set up a global variable to track whether the widgets have been changed
firstTime = True

# create output widget to display filtered DataFrame
output = widgets.Output()
hbox = widgets.HBox([selectTdmVersionWithDate, selectScenarioGroup, selectPercentOrTotal])

# create interactive widget
interactive_output = widgets.interactive_output(update_plot_stackedarea_boardings, {'tdmVersionWithDate': selectTdmVersionWithDate, 'scenarioGroup':selectScenarioGroup, 'percentOrTotal':selectPercentOrTotal})

# Display a markdown heading
display(Markdown('# Trip Mode Split Interactive Chart'))
display(hbox)
display(interactive_output)
display(output)


# Trip Mode Split Interactive Chart

Output()

Output()

In [21]:
dfRouteSummaryWithScenarioData

,scenarioID,tdmVersion,tdmVersionWithDate,scenarioName,scenarioGroup,scenarioYear,MODE,NAMEID,NAME,OPERATOR,...,DY_D8_BRDA,DY_W8_BRDA,DY_8_XITB,DY_D9_BRDA,DY_W9_BRDA,DY_9_XITB,DY_D_BRDA,DY_W_BRDA,DY_BRDA,DY_XITB
0,3.01,v9,WF TDM v9.0 - 2022-12-19,BY_2019,BY,2019,4,60.0,S33,0,...,0.00,0.00,0.00,0.00,0.00,0.00,25.35,877.07,902.42,902.42
1,3.01,v9,WF TDM v9.0 - 2022-12-19,BY_2019,BY,2019,4,61.0,S35,0,...,0.00,0.00,0.00,0.00,0.00,0.00,6.46,46.69,53.15,53.15
2,3.01,v9,WF TDM v9.0 - 2022-12-19,BY_2019,BY,2019,4,63.0,S39,0,...,0.00,0.00,0.00,0.00,0.00,0.00,157.29,1294.65,1451.94,1451.94
3,3.01,v9,WF TDM v9.0 - 2022-12-19,BY_2019,BY,2019,4,65.0,S41,0,...,0.00,0.00,0.00,0.00,0.00,0.00,1.16,40.60,41.76,41.76
4,3.01,v9,WF TDM v9.0 - 2022-12-19,BY_2019,BY,2019,4,48.0,S217,0,...,0.00,0.00,0.00,0.00,0.00,0.00,64.38,1827.24,1891.62,1891.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11070,2.11,v832,WF TDM v8.3.2 - 2022-09-21,NoBuild_2050,NoBuild,2050,4,7.1,M823_5RY2023-,0,...,3.71,7.08,10.79,0.03,1.17,1.20,5.65,138.71,144.36,144.36
11071,2.11,v832,WF TDM v8.3.2 - 2022-09-21,NoBuild_2050,NoBuild,2050,4,1.0,M805_Santaquin,0,...,2.27,59.64,61.91,0.04,14.27,14.31,217.27,169.90,387.17,387.17
11072,2.11,v832,WF TDM v8.3.2 - 2022-09-21,NoBuild_2050,NoBuild,2050,4,12.0,M850_StateStreet,0,...,41.25,225.49,266.74,4.77,423.83,428.60,215.81,2897.04,3112.85,3112.85
11073,2.11,v832,WF TDM v8.3.2 - 2022-09-21,NoBuild_2050,NoBuild,2050,4,15.0,M871_SLtoUtahCnty,0,...,32.77,125.21,157.98,0.00,0.00,0.00,37.22,227.57,264.79,264.79
